# Ứng dụng quản lý menu
(food ID, food name, food price) 
#### Các chức năng chính:

- Tạo/tìm kiếm/chỉnh sửa đơn giá, số lượng món ăn
- Tạo treeview food list
    + Khi thêm 1 món ăn => cập nhật treeview
    + Khi chọn 1 dòng của treeview => show ở các entry
        Xóa
        Sửa: tên và giá
    Quản lý theo ID
- Xuất file excel menu


In [ ]:
import pandas as pd
from tkinter import *
from tkinter import messagebox
from tkinter import ttk
from tkinter.simpledialog import askstring
from tkinter.messagebox import showinfo

### FOOD CLASS
class foods:
    food_id = -1
    foodname = ""
    foodprice = -1
    def __init__(self, _id, _foodname, _foodprice):
        self.food_id = _id
        self.foodname = _foodname
        self.foodprice = _foodprice

# Refresh state for table data and entry values
def food_clear_state():
    food_id.delete(0,"end")
    foodname.delete(0,"end")
    foodprice.delete(0,"end")
    search_id.delete(0,"end")

def refresh_food_data(food_ls):
    for row in foodlist_tree.get_children():
        foodlist_tree.delete(row)
        
    for item in list_foods:
        foodlist_tree.insert(parent='',index='end',values=(item.food_id, item.foodname, item.foodprice))     
        
### Set Food ID when create new food
def set_id(_id_ls):
    tp_id_ls = _id_ls
    for i in range(0, len(tp_id_ls) - 1): 
        for j in range(0, len(tp_id_ls) - 1):
            if tp_id_ls[j] > tp_id_ls[j+1]:
                tp = tp_id_ls[j]
                tp_id_ls[j] = tp_id_ls[j+1]
                tp_id_ls[j+1] = tp
    auto_id = 1
    for item in tp_id_ls:
        if item != auto_id:
            break
        else:
            auto_id = auto_id + 1
    return auto_id

### Add food (id will be generate automation)
def add_food():
    list_id = []
    for item in list_foods:
        list_id.append(item.food_id)
    new_id = set_id(list_id)
    food_id.delete(0, 'end')
    food_id.insert(0, new_id)
    list_foods.append(foods(new_id, foodname.get(), int(foodprice.get())))
    for item in list_foods:
        print(item.food_id, item.foodname, item.foodprice)
    messagebox.showinfo("Adding Food Message", foodname.get() + " added sucessfully")
    foodlist_tree.insert(parent='',index='end',values=(food_id.get(), foodname.get(), foodprice.get()))   
    food_clear_state()     
    
### Search food by ID
def search_food():
    if search_id.get() == '':
        messagebox.showinfo("Notification","Please input Food ID for searching")
    else:
        exist = False  # Giả định item không tồn tại trong list
        id_check = int(search_id.get())
        for item in list_foods:
            if id_check == item.food_id:  # đã có kết quả tìm kiếm
                exist = True  # item đang tìm kiếm có tồn tại trong list
                food_id.delete(0, 'end')
                food_id.insert(0, item.food_id)
                foodname.delete(0, 'end')
                foodname.insert(0, item.foodname)
                foodprice.delete(0, 'end')
                foodprice.insert(0, item.foodprice)
                break
        if exist == False:
            messagebox.showinfo("Notification", f"Food ID : {id_check} does not exist in food menu")
            search_id.delete(0, 'end')
        
### Edit food
def edit_food():
    if search_id.get() == '':
        messagebox.showinfo("Notification","Please input Food ID for editing")
    else:
        edit_foodid = int(search_id.get())
        exist = False
        for i in range(len(list_foods)):
            if edit_foodid == list_foods[i].food_id:
                exist = True
                food_id.delete(0, 'end')
                food_id.insert(0, list_foods[i].food_id)
                foodname.delete(0, 'end')
                foodname.insert(0, list_foods[i].foodname)
                foodprice.delete(0, 'end')
                foodprice.insert(0, list_foods[i].foodprice)
                msg_box = messagebox.askquestion('Message', f"Do you want to edit {edit_foodid} : {list_foods[i].foodname}?", 
                                                 icon='warning')
                if msg_box == 'yes':
                    new_foodprice = askstring('Message', 'Please input the new price') 
                    messagebox.showinfo("Message", f"{edit_foodid} . {list_foods[i].foodname} is edited successfully")
                    foodprice.delete(0, 'end')
                    foodprice.insert(0, new_foodprice)
                    list_foods[i].foodprice = new_foodprice
                    refresh_food_data(tp_food_ls)
                else:
                    messagebox.showinfo('Return', 'You will now return to the application screen')
        if exist == False:
            messagebox.showinfo("Message", f"Food ID : {edit_foodid} does not exist in food menu")
        for item in list_foods:
            print(item.food_id, item.foodname, item.foodprice)

# Delete food
def delete_food():
    if search_id.get() == '':
        messagebox.showinfo("Notification","Please input Food ID to delete")
    else:
        delete_foodid = int(search_id.get())
        exist = False
        for i in range(len(list_foods)):
            if list_foods[i].food_id == delete_foodid:
                exist = True
                tp_name = list_foods[i].foodname
                del list_foods[i]
                messagebox.showinfo("Message", f"{delete_foodid} : {tp_name} is deleted successfully")
                break
        if exist == False:
            messagebox.showinfo("Message", f"{delete_foodid} does not exist in food menu")
        refresh_food_data(tp_food_ls)
        food_clear_state() 
        for item in list_foods:
            print(item.food_id, item.foodname, item.foodprice)
        
# Export Food List to Excel File
def export_xsl():
    tp_foodid_ls = []
    tp_foodname_ls = []
    tp_foodprice_ls = []
    for i in range(len(list_foods)):
        tp_foodid_ls.append(list_foods[i].food_id)
        tp_foodname_ls.append(list_foods[i].foodname)
        tp_foodprice_ls.append(list_foods[i].foodprice)
    col1 = "Food ID"
    col2 = "Food Name"
    col3 = "Price"
    data = pd.DataFrame({col1: tp_foodid_ls, col2:tp_foodname_ls, col3:tp_foodprice_ls})
    data.to_excel('Food List Data.xlsx', sheet_name='Food List', index=False)
    messagebox.showinfo("Notification", "'Food List Data.xlsx' is exported successfully")
    
# Select row on table   
def selectItem(a):
    curItem = foodlist_tree.focus()
    print(foodlist_tree.item(curItem))
    index = foodlist_tree.item(curItem)["values"][0]
    print(index)  #print(type(index))
    food_clear_state()
    for i in range (len(list_foods)):
        if index == list_foods[i].food_id:
            search_id.insert(0, list_foods[i].food_id)
            food_id.insert(0, list_foods[i].food_id)
            foodname.insert(0, list_foods[i].foodname)
            foodprice.insert(0, list_foods[i].foodprice) 
    
### ROOT WINDOW
root= Tk ()
root.title ('Menu Management')
root.geometry("800x400")

title = Label(root, text="MENU MANAGEMENT", font=("arial",20,"bold"), fg="green", bg="yellow")
title.pack(side=TOP, fill=X)

### FOOD LIST
frame_foodls = Frame(root, bd=4, relief=RIDGE, bg="green")
frame_foodls.place(x=0, y=40, width=400, height=460)

title_foodls = Label(frame_foodls, text="FOOD LIST",font=("arial",14,"bold"),bg="green", fg="white")
title_foodls.place(x=120, y=10)

### Treeview Food List
# Food List Frame
tb_foodlist_frame = Frame(frame_foodls, bd=4, relief=RIDGE, bg="white")
tb_foodlist_frame.place(x=50, y=50, width=300, height=250)

# Food List table
foodlist_columns = ('id', 'foodname', 'price')
foodlist_tree = ttk.Treeview(tb_foodlist_frame, columns = foodlist_columns, show = 'headings', selectmode = "browse")

foodlist_tree.heading('id', text='ID')
foodlist_tree.heading('foodname', text='Food Name')
foodlist_tree.heading('price', text='Price')

foodlist_tree.column("# 1",anchor=CENTER, stretch=NO, width = 30)
foodlist_tree.column("# 2",stretch=NO, width = 150)
foodlist_tree.column("# 3",anchor=CENTER, stretch=NO, width = 80)

foodlist_tree.bind('<<TreeviewSelect>>', selectItem)

foodlist_tree.insert('','end',text = 'ID',values = ('ID','Food Name','Price'))

scrollbar_foodlist_tree = Scrollbar(tb_foodlist_frame, orient = VERTICAL, command = foodlist_tree.yview)
foodlist_tree.configure(yscroll = scrollbar_foodlist_tree.set)
scrollbar_foodlist_tree.grid(row = 10, column = 11, sticky = 'ns')
foodlist_tree.grid(row=10, column=8, sticky='nsew', columnspan = 3, pady = 5, padx = 5)

# Button
export_file_btn = Button(frame_foodls, text = 'Export Excel File', width = 20, command=export_xsl)
export_file_btn.place(x=120, y=320)

### DETAIL FOOD FRAME
frame_dtfood = Frame(root, bd=4, relief=RIDGE, bg="light green")
frame_dtfood.place(x=400, y=40, width=400, height=460)

title_dtfood = Label(frame_dtfood, text="Food Detail",font=("arial",14,"bold"),bg="light green", fg="green")
title_dtfood.place(x=15, y=150)

# Label
search_id_lbl = Label(frame_dtfood, text = 'Search ID', font=('calibre',10, 'bold'), bg="light green", fg="black")
search_id_lbl.place(x=15, y=30)
food_id_lbl = Label(frame_dtfood, text = 'ID', font=('calibre',10, 'bold'), bg="light green", fg="black")
food_id_lbl.place(x=15, y=200)
foodname_lbl = Label(frame_dtfood, text = 'Food Name:', font=('calibre',10), bg="light green", fg="black")
foodname_lbl.place(x=15, y=240)
foodprice_lbl = Label(frame_dtfood, text = 'Price:', font=('calibre',10), bg="light green", fg="black")
foodprice_lbl.place(x=15, y=280)

# Etry
search_id = Entry(frame_dtfood)
search_id.place(x=100, y=30)
food_id = Entry(frame_dtfood)
food_id.place(x=100, y=200)
foodname = Entry(frame_dtfood)
foodname.place(x=100, y=240)
foodprice = Entry(frame_dtfood)
foodprice.place(x=100, y=280)

# Button
search_btn = Button(frame_dtfood, text = 'Search', width = 10, command=search_food)
search_btn.place(x=250, y=30)
edit_btn = Button(frame_dtfood, text = 'Edit', width = 10, command=edit_food)
edit_btn.place(x=250, y=70)
tp_delete_btn = Button(frame_dtfood, text = 'Delete', width = 10, command=delete_food)
tp_delete_btn.place(x=250, y=110)
clear_btn = Button(frame_dtfood, text = 'Clear', width = 10, command = food_clear_state)
clear_btn.place(x=250, y=220)
add_btn = Button(frame_dtfood, text = 'Add', width = 10, command=add_food)
add_btn.place(x=250, y=260)

flag_add = False
tp_id_ls = []
list_foods = []
tp_food_ls = list_foods.copy()
refresh_food_data(tp_food_ls)

root.mainloop ()

1 Cá chiên 40
1 Cá chiên 40
2 Thịt heo ram mặn 35
